In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import KFold

In [2]:
X_train = pd.read_csv('./input/X_train.csv')
y_train = pd.read_csv('./input/y_train.csv').iloc[:,0]

In [3]:
# TODO: target transformation

In [6]:
log_rmse = lambda yhat,y: np.sqrt(np.mean((np.log(1+y) - np.log(1+yhat))**2))
def xgboost2perf(learning_rate,max_depth,min_child_weight,gamma,alpha,reg_lambda,\
                 subsample,colsample_bytree,objective,tree_method):

    xgboost = XGBRegressor(learning_rate=learning_rate,\
                           n_estimators=600,\
                           nthread=-1,\
                           
                           max_depth=max_depth,\
                           min_child_weight=min_child_weight,\
                           gamma=gamma,\
                           alpha=alpha,\
                           reg_lambda=reg_lambda,\
                           subsample=subsample,\
                           colsample_bytree=colsample_bytree,\
                           objective=objective,\
                           tree_method=tree_method
                           )

    score = 0
    n_fold = 6
    kf = KFold(n_fold,shuffle=True,random_state=7)
    for train_index, test_index in kf.split(X_train):
        xgboost.fit(X_train.iloc[train_index],y_train.iloc[train_index]);
        yhat = np.maximum(xgboost.predict(X_train.iloc[test_index]),0)
        score += log_rmse(yhat,y_train.iloc[test_index])
    score /= n_fold
    return score

In [7]:
# generate 10 initial random points
bounds = [[0.1,0.6],[2,16],[1.0,100.0],\
          [0.0,100.0],[0.0,100.0],[0.0,100.0],\
          [0.25,1.0],[0.25,1.0],\
          ['reg:squarederror','reg:absoluteerror','reg:gamma','count:poisson'],['exact','hist']]

parameters = []
scores = []
for _ in range(10):
    parameter = []
    for b in bounds:
        if isinstance(b[0],str):
            parameter.append(np.random.choice(b))
        elif isinstance(b[0],int):
            parameter.append(int(b[0]+(b[1]-b[0])*np.random.rand()))
        else:
            parameter.append(b[0]+(b[1]-b[0])*np.random.rand())
    score = xgboost2perf(*parameter)
    scores.append(score)
    parameters.append(parameter)

parameters = pd.DataFrame(parameters,columns=['learning_rate', 'max_depth', 'min_child_weight', 'gamma', 'alpha', 'reg_lambda', 'subsample', 'colsample_bytree', 'objective', 'tree_method'])
parameters.to_csv('./input/parameters.csv',index=False)
scores = pd.DataFrame(scores,columns=['scores'])
scores.to_csv('./input/scores.csv')